In [1]:
import torch
import torch_geometric
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm

sys.path.append('../')

from torch_geometric.data import Data
from torch_geometric.contrib.nn.models import GRBCDAttack
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import NormalizeFeatures
from engine.model.GCN import SimpleGCN

c:\Users\shyun\AppData\Local\anaconda3\envs\lig2023\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\shyun\AppData\Local\anaconda3\envs\lig2023\Lib\site-packages\torch_geometric\contrib\__init__.py:8: UserWarning: 'torch_geometric.contrib' contains experimental code and is subject to change. Please use with caution.
  warnings.warn(


In [6]:
dataName = 'Cora'
dataset = Planetoid(f'./{dataName}', dataName, pre_transform=NormalizeFeatures())[0]

In [7]:
adj = np.load('../data/dataset/cora_adj.npz')
col = adj['col']
row = adj['row']
val = adj['value']
pdeg=  [0 for x in range(len(dataset.x))]

for e, _ in enumerate(col):
    pdeg[col[e]] = pdeg[col[e]] + val[e]
    pdeg[row[e]] = pdeg[row[e]] + val[e]

deg = [0 for x in range(len(dataset.x))]

for e, _ in enumerate(dataset.edge_index[0]):
    deg[dataset.edge_index[0][e]] = deg[dataset.edge_index[0][e]] + 1
    deg[dataset.edge_index[1][e]] = deg[dataset.edge_index[1][e]] + 1

In [8]:
prate = [(pdeg[i] / deg[i]) if deg[i] != 0  else 0 for i in range(len(dataset.x))]

In [9]:
import pickle
with open(f'../result/pickles/{dataName}Budget.pkl', 'rb') as f:
    budget = pickle.load(f)


In [10]:
pfeat = []
pbud = []
budList = []
dist = []
mv = 0
for b in budget:
    if b not in budList:
        budList.append(b)
budList.sort()
dist = [[] for d in budList]
for i, b in enumerate(budget):
    if b > 0:
        pbud.append(b)
        pfeat.append(prate[i])
        dist[budList.index(b)].append(prate[i])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for i, d in enumerate(dist):
    for v in d:
        plt.scatter(v,budList[i], c = 'b', alpha = 1 / len(d))
title = f'{dataName} iid rate and budget'
plt.title(title)
plt.savefig(f'../result/plot/{title}.png')
plt.show()